In [1]:
import matplotlib.pyplot as plt
from atomtoolkit import *
from atomtoolkit import IO
import atomtoolkit.render.lineshapes as lineshapes
from atomtoolkit.atom import Atom
import atomtoolkit.render.spectra as spectra
from atomtoolkit.render.grotrian import draw_levels
from pprint import pprint
import numpy as np

In [2]:
resource_path = 'C:/Users/jippi/PycharmProjects/Atom-Toolkit/resources'

def load_atom(species, num_levels=None, save=False, B=Q_(0.0, 'G'), load=False):
    if load:
        a = Atom.load(f'atoms/{species}.atom')
    else:
        df = IO.load_NIST_data(speciesdict[species]['species'])
        display(df)
        if 'trans_path' in speciesdict[species]:
            trans_df = IO.load_transition_data(speciesdict[species]['trans_path'], columns={
                "conf_l": "LConfiguration", "conf_u": "UConfiguration",
                "term_l": "LTerm", "term_u": "UTerm", "j_l": "LJ", "j_u": "UJ",
                "A": "A DREAM"}).dropna(subset=['A'])
        else:
            trans_df = None
        a = IO.generate_full_from_dataframe(df, species, speciesdict[species]['I'],
                                            num_levels=num_levels,
                                            B=B,
                                            hf_csv=f'{resource_path}/{species}_Hyperfine.csv',
                                            transitions_df=trans_df,
                                            allowed=0b101)
        if save:
            a.save(f'atoms/{species}.atom')
            a.generate_hf_csv(filename=f'{resource_path}/{species}_Hyperfine.csv')
        print('Generation complete')
    return a

In [3]:
speciesdict = {
    '171Yb': {'species': 'Yb II', 'I': 0.5, 'trans_path': f"{resource_path}/Yb_II_Oscillator_Strengths.csv"},
    '171Yb_2': {'species': 'Yb II', 'I': 0.5},
    '173Yb': {'species': 'Yb II', 'I': 2.5, 'trans_path': f"{resource_path}/Yb_II_Oscillator_Strengths.csv"},
    '138Ba': {'species': 'Ba II', 'I': 0.0},
    '133Ba': {'species': 'Ba II', 'I': 0.5},
    '201Hg': {'species': 'Hg I', 'I': 1.5},
    '9Be': {'species': 'Be II', 'I': 1.5}
}

In [4]:
# whether to load from pickle
load = False
# whether to save the ion
save = False
# Name of the atom
species = '173Yb'
# Number of levels to generate
num_levels = 20
# Magnetic field
B = Q_(5.0, 'G')

a = load_atom(species, load=load, save=save, num_levels=num_levels, B=B)

data finalized: 100%|████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  6.71it/s]


,Configuration,Term,J,Level (cm-1),Lande,Percentage,Percentage_2,Configuration_2,Term_2,Level (Hz)
0,4f14.6s,2S,1/2,0.0,1.998,100.0,NaN,None,NaN,0.0
1,4f13.(2F*).6s2,2F*,7/2,21418.75,1.145,98.0,1.0,4f13.(2F*<7/2>).5d2.(1S),1[7/2]*,642117970978750.0
2,4f13.(2F*).6s2,2F*,5/2,31568.08,0.862,90.0,7.0,4f13.(2F*<7/2>)5d.6s.(3D),3[7/2]*,946387229754064.0
3,4f14.5d,2D,3/2,22960.8,1.802,100.0,NaN,None,NaN,688347466964640.0
4,4f14.5d,2D,5/2,24332.69,1.202,100.0,NaN,None,NaN,729475694485202.0
...,...,...,...,...,...,...,...,...,...,...
328,,NaN,11/2,96382.06,1.230,100.0,NaN,None,NaN,2889461467450348.0
329,,NaN,9/2,96482.44,NaN,100.0,NaN,None,NaN,2892470784143752.0
330,,NaN,11/2,96898.03,NaN,100.0,NaN,None,NaN,2904929858905774.0
331,,NaN,13/2,97043.28,NaN,100.0,NaN,None,NaN,2909284344358224.0


adding level 4f13.(2F*<7/2>).5d.6s.(3D) 3[9/2]*7/2                                                                     
adding transition 4f14.5d 2D3/2 → 4f14.6p 2P*1/2                                                                       
adding internal transitions to 4f13.(2F*<7/2>).5d.6s.(3D) 3[9/2]*7/2                                                   

Generation complete


In [5]:
# pprint(a.levels.list_names())

In [7]:
# pprint(a.transitions.list_names())

In [8]:
a.transitions[('4f14.5d 2D3/2', '4f13.(2F*<7/2>).5d.6s.(3D) 3[3/2]*1/2')].set_frequency(Q_(320.570339, 'THz'))
a.transitions[('4f14.6s 2S1/2', '4f14.6p 2P*1/2')].set_frequency(Q_(812.128235, 'THz'))

In [6]:
# plt.figure(figsize=(20, 10))
# draw_levels(a)

In [11]:
a.compute_branching_ratios('4f14.6p 2P*3/2')

{'4f14.6s 2S1/2': 0.987698986975398,
 '4f14.5d 2D3/2': 0.002098408104196816,
 '4f14.5d 2D5/2': 0.010202604920405211}

In [38]:
a.linked_levels('4f14.6p 2P*3/2')

{'4f14.6s 2S1/2': Transition(4f14.6s 2S1/2 → 4f14.6p 2P*3/2, freq=911136133580133.9 hertz, A=136.5,
 '4f14.5d 2D3/2': Transition(4f14.5d 2D3/2 → 4f14.6p 2P*3/2, freq=222788666615493.88 hertz, A=0.29,
 '4f14.5d 2D5/2': Transition(4f14.5d 2D5/2 → 4f14.6p 2P*3/2, freq=181660439094931.88 hertz, A=1.41}

In [39]:
a.transitions[('4f14.6p 2P*3/2', '4f14.5d 2D3/2')].wl

1345.6360350565105 <Unit('nanometer')>

In [40]:
a.transitions[('4f14.6p 2P*3/2', '4f14.6s 2S1/2')].wl

329.0314662662134 <Unit('nanometer')>

In [41]:
a.levels['4f14.6p 2P*3/2'].hfA.to('MHz')

875.4 <Unit('megahertz')>

In [2]:
import pandas as pd
pd.set_option("max_colwidth", 80)
pd.set_option("max_rows", 80)

df = IO.load_NIST_data('Yb II')

from atomtoolkit.atom import MultiTerm

M = MultiTerm.from_dataframe(df, 2)

M.__dict__

data finalized: 100%|████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 10.05it/s]


{'terms': [Term(4f13.(2F*).6s2 2F*5/2),
  Term(4f13.(2F*<7/2>)5d.6s.(3D) 3[7/2]*5/2)],
 'terms_dict': {90.0: Term(4f13.(2F*).6s2 2F*5/2),
  7.0: Term(4f13.(2F*<7/2>)5d.6s.(3D) 3[7/2]*5/2)}}

In [25]:

import re
def reconstitute_conf(c0, c1):
    # grab any info in c0 that has been removed from c1 and put it back where it belongs
    if not c1:
        return
    if re.match(r'\(.+?\)\(.+?\)', c1):
        return re.sub( r'\((.+?)\)', '{}',c0).format(*re.findall(r'\((.+?)\)', c1))
    elif re.match(r'\(.+?\).+\(.+?\)', c1):
        return c0.split('(')[0]+c1
    else:
        return c1

In [26]:
reconstitute_conf('4f13.(2F*<7/2>).5d.6s.(3D)', None)